In [12]:
# Imports
import pandas as pd
from sqlalchemy import create_engine

In [13]:
# Upload data for admit_details and convert to pandas dataframe

admit_file = 'data/full_hospital_detail.csv'

admit_details_df = pd.read_csv(admit_file, na_filter = True, na_values = '[]')
admit_details_df.rename(columns = {'Location': 'hospital_code', 'DOS' : 'date', 'Decision to Discharge Time (min)':'decision_to_discharge_time'}, inplace = True)

admit_details_df.head()

,hospital_code,date,Disposition,decision_to_discharge_time
0,A,1/1/2021,Admitted,54.0
1,A,1/1/2021,Admitted,51.0
2,A,1/1/2021,Admitted,69.0
3,A,1/1/2021,Admit Obs,112.0
4,A,1/1/2021,Admitted,122.0


In [14]:
# Upload data for hospitals and convert to pandas dataframe

hospital_file = 'data/hospitals.csv'

hospitals_df = pd.read_csv(hospital_file, na_filter = True, na_values = '[]')
hospitals_df = hospitals_df[['Hospital', 'City', 'Latitude', 'Longitude']]
hospitals_df.rename(columns = {'Hospital': 'hospital_code'}, inplace = True)


hospitals_df.head()


,hospital_code,City,Latitude,Longitude
0,A,Bemidji,47.478542,-94.890787
1,B,Warroad,48.905266,-95.314404
2,C,Mora,45.876903,-93.293835
3,D,Saint Paul,44.949749,-93.093103
4,E,Minneapolis,44.977300,-93.265469


In [15]:
## Updading the code to be SQLite
import sqlite3

engine = create_engine("sqlite:///Hospital_Boarding.sqlite")
conn = sqlite3.connect('Hospital_Boarding.sqlite')
#"sqlite:///Hospital_Boarding.db"

print("Opened database successfully")


Opened database successfully


In [16]:

c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS hospitals''')

c.execute('''CREATE TABLE hospitals(
    hospital_code CHAR(1) PRIMARY KEY  NOT NULL,
    City VARCHAR   NOT NULL,
    Latitude DECIMAL   NOT NULL,
    Longitude DECIMAL   NOT NULL)
    '''
)

c.execute('''DROP TABLE IF EXISTS admit_details''')

c.execute('''CREATE TABLE admit_details (
    record_id INTEGER PRIMARY KEY   NOT NULL,
    hospital_code CHAR(1)   NOT NULL,
    date VARCHAR   NOT NULL,
    disposition VARCHAR   NOT NULL,
    decision_to_discharge_time INTEGER
     )'''
)

# c.execute('''ALTER TABLE admit_details ADD CONSTRAINT fk_admit_details_hospital_code FOREIGN KEY(hospital_code)
# REFERENCES hospitals (hospital_code)'''
# )

print("Table created successfully")

#conn.commit()

#conn.close()

Table created successfully


In [17]:
data=c.execute('''SELECT * FROM hospitals''')
print(data.description)

(('hospital_code', None, None, None, None, None, None), ('City', None, None, None, None, None, None), ('Latitude', None, None, None, None, None, None), ('Longitude', None, None, None, None, None, None))


In [18]:
data2=c.execute('''SELECT * FROM admit_details''')
print(data2.description)

(('record_id', None, None, None, None, None, None), ('hospital_code', None, None, None, None, None, None), ('date', None, None, None, None, None, None), ('disposition', None, None, None, None, None, None), ('decision_to_discharge_time', None, None, None, None, None, None))


In [19]:
# connect to local database

# protocol = 'postgresql'
# username = '<username>'
# password = '<password>'
# host = 'localhost'
# port = 5432
# database_name = 'Hospital_Boarding'
# connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'


#engine = create_engine(connection_string)

In [20]:
# # Preview table names

# engine.table_names()

In [21]:
# Use pandas to load DataFrame into database

hospitals_df.to_sql('hospitals', con=engine, if_exists='append', index=False)
admit_details_df.to_sql('admit_details', con=engine, if_exists='append', index=False)


In [22]:
#  Close the coonection
conn.close()